In [1]:
import pandas as pd
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
from scipy.optimize import minimize
import scipy
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.1/Resources/'
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import datetime
from rpy2.robjects.vectors import FloatVector
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression

real_data_folder = "/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Data/sp500_mixed_period_2000_2021/"
np.random.seed(881)

# Define the model that generates pair simulations.
yuima = importr("yuima")
n_ou_sim_string = """
n_sim_ou = function(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length){

  set.seed(random_seed)

  drift = c("mu11*t-mu12*X1", "mu21*t-mu22*X2")
  diffusion = matrix(c("exp(sigma11)", "exp(sigma12)", "exp(sigma21)", "exp(sigma22)"), 2, 2, byrow=TRUE)
  ou_model = setModel(drift=drift, diffusion=diffusion, 
                        time.variable = "t",
                        state.var=c("X1","X2"), solve.variable=c("X1","X2"))

  newsamp = setSampling(Initial=T0, Terminal=T, n=length)

  n_sim_ou_data = data.frame(matrix(nrow=length+1, ncol=2*num_sim))
  for (i in 1:num_sim){
    ou_sim = simulate(ou_model, 
                      true.par=list(
                        mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22, 
                        sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22), 
                      xinit=xinit_vec[i], sampling=newsamp)
    original_data = ou_sim@data@original.data
    one_sim_ou = data.frame(original_data[,1], original_data[,2])
    colnames(one_sim_ou) = c('series1', 'series2')
    n_sim_ou_data[, (2*i-1):(2*i)] = one_sim_ou
  }
  return(n_sim_ou_data)
}
"""

n_ou_sim = SignatureTranslatedAnonymousPackage(n_ou_sim_string, "n_ou_sim")
def n_ou_simulation(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length):
    """num_sim simulations of bivariate Ornstein-Uhlenbeck process,
    length = length of one series
    """
    n_ou_sim_data = pd.DataFrame(
        n_ou_sim.n_sim_ou(random_seed=random_seed, num_sim=num_sim,
                              mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22,
                              sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22,
                              xinit_vec=xinit_vec, T0=T0, T=T, length=length)).transpose()
    return n_ou_sim_data

def price_to_log_price(n_price):
    return(np.log(n_price))

def log_price_to_price(n_log_price):
    return(np.exp(n_log_price))

def price_to_return(n_price):
    n_return = pd.DataFrame()
    for i in range(n_price.shape[1]):
        ith_column_price_series = n_price.iloc[:, i]
        n_return = pd.concat([n_return, 100 * (np.log(ith_column_price_series[1:].values) - np.log(ith_column_price_series[:-1]))], axis=1)
    return n_return

def log_price_to_return(n_log_price):
    n_real_return = pd.DataFrame()
    for i in range(n_log_price.shape[1]):
        ith_column_price_series = n_log_price.iloc[:, i]
        n_real_return = pd.concat([n_real_return, 100 * (ith_column_price_series[1:].values - ith_column_price_series[:-1])], axis=1)
    return n_real_return


def cross_corr_coef(lag_time_series, lead_time_series, lag):
    # calculate the cross correlation between two time series
    # if the result is not zero, then 'lead_time_series' leads 'lag_time_series'
    lag_time_series = lag_time_series.iloc[lag:]
    corr_coef = np.corrcoef(lag_time_series, lead_time_series.iloc[0:((lead_time_series.size)-lag)])[0][1]
    return corr_coef


def cal_stats(n_return, n_price):
    
    return_series1 = n_return.iloc[:, ::2]
    return_series2 = n_return.iloc[:, 1::2]
    price_series1 = n_price.iloc[:, ::2]
    price_series2 = n_price.iloc[:, 1::2]
    sq_return_series1 = return_series1**2
    sq_return_series2 = return_series2**2

    
    mean1 = return_series1.mean(axis=0).values
    sd1 = return_series1.std(axis=0).values
    skew1 = return_series1.skew(axis=0).values
    kurtosis1 = return_series1.kurtosis(axis=0).values
       
    
    mean2 = return_series2.mean(axis=0).values
    sd2 = return_series2.std(axis=0).values
    skew2 = return_series2.skew(axis=0).values
    kurtosis2 = return_series2.kurtosis(axis=0).values #8(8)
    
    
    
    # Autocorrelation of the 2 return series with 3 lags respectively
    autocorr1_lag_1 = return_series1.apply(lambda x: x.autocorr(lag=1))
    autocorr1_lag_2 = return_series1.apply(lambda x: x.autocorr(lag=2))
    autocorr1_lag_3 = return_series1.apply(lambda x: x.autocorr(lag=3))
    autocorr2_lag_1 = return_series2.apply(lambda x: x.autocorr(lag=1)) 
    autocorr2_lag_2 = return_series2.apply(lambda x: x.autocorr(lag=2)) 
    autocorr2_lag_3 = return_series2.apply(lambda x: x.autocorr(lag=3)) #6(14)
    
    # Cross-correlation between return series
    corr_ts1_lag_0 = []
    corr_ts1_lag_1 = [] # if not zero, return series 2 leads return series 1
    corr_ts1_lag_2 = []
    corr_ts1_lag_3 = []
    corr_ts2_lag_1 = [] # if not zero, return series 1 lead return series 2
    corr_ts2_lag_2 = []
    corr_ts2_lag_3 = []
    for i in range(int(real_price.shape[1]/2)):
        corr_ts1_lag_0.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 0))
        corr_ts1_lag_1.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 1))
        corr_ts1_lag_2.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 2))
        corr_ts1_lag_3.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 3))
        corr_ts2_lag_1.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 1))
        corr_ts2_lag_2.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 2))
        corr_ts2_lag_3.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 3))
    corr_ts1_lag_0 = pd.Series(corr_ts1_lag_0)
    corr_ts1_lag_1 = pd.Series(corr_ts1_lag_1)
    corr_ts1_lag_2 = pd.Series(corr_ts1_lag_2)
    corr_ts1_lag_3 = pd.Series(corr_ts1_lag_3)
    corr_ts2_lag_1 = pd.Series(corr_ts2_lag_1)
    corr_ts2_lag_2 = pd.Series(corr_ts2_lag_2)
    corr_ts2_lag_3 = pd.Series(corr_ts2_lag_3)#7(21)
    
    
    
    # Autocorrelation of the 2 squared returns with 3 lags respectively
    autocorr_sqreturn_1_lag_1 = sq_return_series1.apply(lambda x: x.autocorr(lag=1))
    autocorr_sqreturn_1_lag_2 = sq_return_series1.apply(lambda x: x.autocorr(lag=2))
    autocorr_sqreturn_1_lag_3 = sq_return_series1.apply(lambda x: x.autocorr(lag=3))
    autocorr_sqreturn_2_lag_1 = sq_return_series2.apply(lambda x: x.autocorr(lag=1)) 
    autocorr_sqreturn_2_lag_2 = sq_return_series2.apply(lambda x: x.autocorr(lag=2)) 
    autocorr_sqreturn_2_lag_3 = sq_return_series2.apply(lambda x: x.autocorr(lag=3))#6(27)
    
    # Cross-correlation between squared returns series
    corr_sqts1_lag_0 = []
    corr_sqts1_lag_1 = [] # if not zero, return series 2 leads return series 1
    corr_sqts1_lag_2 = []
    corr_sqts1_lag_3 = []
    corr_sqts2_lag_1 = [] # if not zero, return series 1 lead return series 2
    corr_sqts2_lag_2 = []
    corr_sqts2_lag_3 = []
    for i in range(int(real_price.shape[1]/2)):
        corr_sqts1_lag_0.append(cross_corr_coef(sq_return_series1.iloc[:, i], sq_return_series2.iloc[:, i], 0))
        corr_sqts1_lag_1.append(cross_corr_coef(sq_return_series1.iloc[:, i], sq_return_series2.iloc[:, i], 1))
        corr_sqts1_lag_2.append(cross_corr_coef(sq_return_series1.iloc[:, i], sq_return_series2.iloc[:, i], 2))
        corr_sqts1_lag_3.append(cross_corr_coef(sq_return_series1.iloc[:, i], sq_return_series2.iloc[:, i], 3))
        corr_sqts2_lag_1.append(cross_corr_coef(sq_return_series2.iloc[:, i], sq_return_series1.iloc[:, i], 1))
        corr_sqts2_lag_2.append(cross_corr_coef(sq_return_series2.iloc[:, i], sq_return_series1.iloc[:, i], 2))
        corr_sqts2_lag_3.append(cross_corr_coef(sq_return_series2.iloc[:, i], sq_return_series1.iloc[:, i], 3))
    corr_sqts1_lag_0 = pd.Series(corr_ts1_lag_0)
    corr_sqts1_lag_1 = pd.Series(corr_ts1_lag_1)
    corr_sqts1_lag_2 = pd.Series(corr_ts1_lag_2)
    corr_sqts1_lag_3 = pd.Series(corr_ts1_lag_3)
    corr_sqts2_lag_1 = pd.Series(corr_ts2_lag_1)
    corr_sqts2_lag_2 = pd.Series(corr_ts2_lag_2)
    corr_sqts2_lag_3 = pd.Series(corr_ts2_lag_3)#7(34)
    
    
    
    
    stats_data = pd.DataFrame([mean1, mean2, sd1, sd2, 
                               skew1, skew2, kurtosis1, kurtosis2, #8(8)
                               
                               autocorr1_lag_1, autocorr1_lag_2, autocorr1_lag_3,
                               autocorr2_lag_1, autocorr2_lag_2, autocorr2_lag_3,#6(14)
                               
                               corr_ts1_lag_0, 
                               corr_ts1_lag_1, corr_ts1_lag_2, corr_ts1_lag_3,
                               corr_ts2_lag_1, corr_ts2_lag_2, corr_ts2_lag_3, #7(21)
                               
                               autocorr_sqreturn_1_lag_1, autocorr_sqreturn_1_lag_2, autocorr_sqreturn_1_lag_3,
                               autocorr_sqreturn_2_lag_1, autocorr_sqreturn_2_lag_2, autocorr_sqreturn_2_lag_3,#6(27)
                               
                               corr_sqts1_lag_0,
                               corr_sqts1_lag_1, corr_sqts1_lag_2, corr_sqts1_lag_3,
                               corr_sqts2_lag_1, corr_sqts2_lag_2, corr_sqts2_lag_3])#7(34)
    stats_data = stats_data.transpose()
    stats_data.columns = [
        'return_mean1', 'return_mean2',
        'return_sd1', 'return_sd2',
        'return_skew1', 'return_skew2',
        'return_kurtosis1', 'return_kurtosis2',
        
        'return_autocorrelation_ts1_lag1', 'return_autocorrelation_ts1_lag2', 'return_autocorrelation_ts1_lag3',
        'return_autocorrelation_ts2_lag1', 'return_autocorrelation_ts2_lag2', 'return_autocorrelation_ts2_lag3',
        
        'return_correlation_ts1_lag_0',
        'return_correlation_ts1_lag_1', 'return_correlation_ts1_lag_2', 'return_correlation_ts1_lag_3',
        'return_correlation_ts2_lag_1', 'return_correlation_ts2_lag_2', 'return_correlation_ts2_lag_3',
        
        'sqreturn_autocorrelation_ts1_lag1', 'sqreturn_autocorrelation_ts1_lag2', 'sqreturn_autocorrelation_ts1_lag3',
        'sqreturn_autocorrelation_ts2_lag1', 'sqreturn_autocorrelation_ts2_lag2', 'sqreturn_autocorrelation_ts2_lag3',
    
        'sqreturn_correlation_ts1_lag_0',
        'sqreturn_correlation_ts1_lag_1', 'sqreturn_correlation_ts1_lag_2', 'sqreturn_correlation_ts1_lag_3', 
        'sqreturn_correlation_ts2_lag_1', 'sqreturn_correlation_ts2_lag_2', 'sqreturn_correlation_ts2_lag_3']
    
    return stats_data


def loss_function(params):

    params = FloatVector(params)
    print(params)
    moment_loss = pd.DataFrame().reindex_like(real_stats)

    n_real_log_price = n_ou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=mu11, mu12=mu12, 
        mu21=mu21, mu22=mu22,
        sigma11=sigma11, sigma12=sigma12, 
        sigma21=sigma21, sigma22=sigma22,
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    n_real_price = log_price_to_price(n_real_log_price)
    n_real_return = price_to_return(n_real_price)
    n_real_stats = cal_stats(n_real_return, n_real_price)
    
    
    n_sim_log_price = n_ou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=params[0], mu12=params[1], 
        mu21=params[2], mu22=params[3],
        sigma11=params[4], sigma12=params[5], 
        sigma21=params[6], sigma22=params[7],
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    n_sim_price = log_price_to_price(n_sim_log_price)
    n_sim_return = price_to_return(n_sim_price)
    n_sim_stats = cal_stats(n_sim_return, n_sim_price)


    moment_loss = np.abs(n_real_stats - n_sim_stats)
    sum_loss = np.sum(moment_loss)
    
    sum_loss[6] = sum_loss[6]/10 
    sum_loss[7] = sum_loss[7]/10 
    
    loss = np.sum(sum_loss)
    
    print(sum_loss)
    print(loss)
    print('----------')

    return loss



mu11 = 0.5689102
mu12 = 0.1308214 
mu21 = 0.6956711 
mu22 = 0.1349717
sigma11 = -3.3933278
sigma12 = -1.2515663 
sigma21 = -1.3669886
sigma22 = -2.9761180   


real_price = pd.read_csv(real_data_folder + "pair_prices_cal.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv(real_data_folder + "pair_returns_cal.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=real_price)


xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
num_sim, T0, T, length = real_stats.shape[0], 0, 2, real_price.shape[0]


initial0 = [1, 1, 1, 1, -1, -1, -1, -1]
begin_time = datetime.datetime.now()
res = minimize(loss_function, initial0, method='Powell',
               tol=1e-6, options={'disp': True},
               bounds=[(0, None), (0, None), (0, None), (0, None),
                       (None, None), (None, None), (None, None), (None, None)])
print(res.x)

time = datetime.datetime.now() - begin_time
print(time)

params = (res.x)
loss = loss_function((params))
print(loss)

[1]  1  1  1  1 -1 -1 -1 -1

return_mean1                         105.647953
return_mean2                         110.856026
return_sd1                           372.363287
return_sd2                           415.042301
return_skew1                          33.955207
return_skew2                          31.939340
return_kurtosis1                       5.988442
return_kurtosis2                       5.754894
return_autocorrelation_ts1_lag1       11.879293
return_autocorrelation_ts1_lag2       13.803541
return_autocorrelation_ts1_lag3       13.084647
return_autocorrelation_ts2_lag1       12.287808
return_autocorrelation_ts2_lag2       14.405159
return_autocorrelation_ts2_lag3       13.740927
return_correlation_ts1_lag_0         170.708240
return_correlation_ts1_lag_1          12.457401
return_correlation_ts1_lag_2          12.724620
return_correlation_ts1_lag_3          13.468655
return_correlation_ts2_lag_1          11.895863
return_correlation_ts2_lag_2          13.788204
return_corr

return_mean1                          39.205432
return_mean2                         108.450106
return_sd1                           371.790833
return_sd2                           410.468581
return_skew1                          28.577826
return_skew2                          30.200958
return_kurtosis1                       6.039867
return_kurtosis2                       5.924524
return_autocorrelation_ts1_lag1       13.966585
return_autocorrelation_ts1_lag2       13.648197
return_autocorrelation_ts1_lag3       12.956808
return_autocorrelation_ts2_lag1       12.048397
return_autocorrelation_ts2_lag2       11.227549
return_autocorrelation_ts2_lag3       12.825486
return_correlation_ts1_lag_0         169.808479
return_correlation_ts1_lag_1          12.757643
return_correlation_ts1_lag_2          12.706565
return_correlation_ts1_lag_3          12.214491
return_correlation_ts2_lag_1          12.884289
return_correlation_ts2_lag_2          12.202019
return_correlation_ts2_lag_3          12

return_mean1                          38.185028
return_mean2                         106.909384
return_sd1                           376.840268
return_sd2                           415.912174
return_skew1                          30.943779
return_skew2                          29.158622
return_kurtosis1                       5.915308
return_kurtosis2                       6.008041
return_autocorrelation_ts1_lag1       13.572808
return_autocorrelation_ts1_lag2       13.226902
return_autocorrelation_ts1_lag3       13.638330
return_autocorrelation_ts2_lag1       13.833152
return_autocorrelation_ts2_lag2       12.153158
return_autocorrelation_ts2_lag3       12.517842
return_correlation_ts1_lag_0         168.189882
return_correlation_ts1_lag_1          13.516690
return_correlation_ts1_lag_2          12.823772
return_correlation_ts1_lag_3          12.758934
return_correlation_ts2_lag_1          13.053023
return_correlation_ts2_lag_2          12.812113
return_correlation_ts2_lag_3          13

return_mean1                          39.438077
return_mean2                         108.044689
return_sd1                           376.256053
return_sd2                           414.053900
return_skew1                          29.669991
return_skew2                          30.899192
return_kurtosis1                       5.732180
return_kurtosis2                       5.980401
return_autocorrelation_ts1_lag1       14.081110
return_autocorrelation_ts1_lag2       13.033560
return_autocorrelation_ts1_lag3       12.822884
return_autocorrelation_ts2_lag1       12.842097
return_autocorrelation_ts2_lag2       12.798957
return_autocorrelation_ts2_lag3       12.685562
return_correlation_ts1_lag_0         169.945905
return_correlation_ts1_lag_1          13.760805
return_correlation_ts1_lag_2          13.129460
return_correlation_ts1_lag_3          12.227994
return_correlation_ts2_lag_1          13.175136
return_correlation_ts2_lag_2          12.609968
return_correlation_ts2_lag_3          13

return_mean1                          41.799351
return_mean2                         110.806216
return_sd1                           375.238824
return_sd2                           413.147631
return_skew1                          29.092104
return_skew2                          30.611042
return_kurtosis1                       5.744872
return_kurtosis2                       5.483617
return_autocorrelation_ts1_lag1       13.068926
return_autocorrelation_ts1_lag2       13.466840
return_autocorrelation_ts1_lag3       12.631174
return_autocorrelation_ts2_lag1       12.290503
return_autocorrelation_ts2_lag2       12.872238
return_autocorrelation_ts2_lag3       11.892607
return_correlation_ts1_lag_0         169.369335
return_correlation_ts1_lag_1          12.645125
return_correlation_ts1_lag_2          14.170115
return_correlation_ts1_lag_3          12.600536
return_correlation_ts2_lag_1          12.506999
return_correlation_ts2_lag_2          12.427159
return_correlation_ts2_lag_3          13

...

return_mean1                         23.865711
return_mean2                         30.682988
return_sd1                           17.144244
return_sd2                           15.292454
return_skew1                         31.125745
return_skew2                         30.964275
return_kurtosis1                      6.084224
return_kurtosis2                      5.985736
return_autocorrelation_ts1_lag1      14.908502
return_autocorrelation_ts1_lag2      14.901791
return_autocorrelation_ts1_lag3      15.299067
return_autocorrelation_ts2_lag1      12.876103
return_autocorrelation_ts2_lag2      13.293344
return_autocorrelation_ts2_lag3      12.933291
return_correlation_ts1_lag_0         11.895059
return_correlation_ts1_lag_1         14.118256
return_correlation_ts1_lag_2         12.816756
return_correlation_ts1_lag_3         11.416066
return_correlation_ts2_lag_1         12.971434
return_correlation_ts2_lag_2         12.398193
return_correlation_ts2_lag_3         12.260786
sqreturn_auto

return_mean1                         22.180960
return_mean2                         28.789677
return_sd1                           18.960387
return_sd2                           20.990549
return_skew1                         29.679759
return_skew2                         31.833614
return_kurtosis1                      6.203551
return_kurtosis2                      5.837946
return_autocorrelation_ts1_lag1      14.208757
return_autocorrelation_ts1_lag2      14.837044
return_autocorrelation_ts1_lag3      14.111604
return_autocorrelation_ts2_lag1      14.061209
return_autocorrelation_ts2_lag2      12.293318
return_autocorrelation_ts2_lag3      12.388201
return_correlation_ts1_lag_0         35.405530
return_correlation_ts1_lag_1         12.913358
return_correlation_ts1_lag_2         12.258937
return_correlation_ts1_lag_3         13.759312
return_correlation_ts2_lag_1         12.651494
return_correlation_ts2_lag_2         12.723046
return_correlation_ts2_lag_3         12.783822
sqreturn_auto

return_mean1                         23.536840
return_mean2                         29.044876
return_sd1                           16.776763
return_sd2                           15.294280
return_skew1                         29.903990
return_skew2                         30.181648
return_kurtosis1                      5.935402
return_kurtosis2                      5.879781
return_autocorrelation_ts1_lag1      15.281147
return_autocorrelation_ts1_lag2      14.444137
return_autocorrelation_ts1_lag3      14.444147
return_autocorrelation_ts2_lag1      12.048388
return_autocorrelation_ts2_lag2      13.248155
return_autocorrelation_ts2_lag3      11.621919
return_correlation_ts1_lag_0         13.123526
return_correlation_ts1_lag_1         12.833137
return_correlation_ts1_lag_2         12.835099
return_correlation_ts1_lag_3         12.999011
return_correlation_ts2_lag_1         11.986187
return_correlation_ts2_lag_2         13.278634
return_correlation_ts2_lag_3         14.116470
sqreturn_auto

return_mean1                         23.069703
return_mean2                         29.736029
return_sd1                           18.104832
return_sd2                           17.314759
return_skew1                         28.622498
return_skew2                         29.559713
return_kurtosis1                      6.420841
return_kurtosis2                      6.311950
return_autocorrelation_ts1_lag1      15.164788
return_autocorrelation_ts1_lag2      15.655151
return_autocorrelation_ts1_lag3      14.966541
return_autocorrelation_ts2_lag1      11.995288
return_autocorrelation_ts2_lag2      13.074552
return_autocorrelation_ts2_lag3      11.823791
return_correlation_ts1_lag_0         12.553407
return_correlation_ts1_lag_1         12.598290
return_correlation_ts1_lag_2         13.006080
return_correlation_ts1_lag_3         12.946469
return_correlation_ts2_lag_1         12.274917
return_correlation_ts2_lag_2         12.562534
return_correlation_ts2_lag_3         13.428156
sqreturn_auto

return_mean1                         22.877353
return_mean2                         29.048316
return_sd1                           17.660404
return_sd2                           16.594368
return_skew1                         31.703157
return_skew2                         28.444745
return_kurtosis1                      6.048054
return_kurtosis2                      6.218525
return_autocorrelation_ts1_lag1      13.934829
return_autocorrelation_ts1_lag2      15.043964
return_autocorrelation_ts1_lag3      14.709230
return_autocorrelation_ts2_lag1      11.909513
return_autocorrelation_ts2_lag2      12.710066
return_autocorrelation_ts2_lag3      12.639442
return_correlation_ts1_lag_0         13.018793
return_correlation_ts1_lag_1         12.186244
return_correlation_ts1_lag_2         13.031126
return_correlation_ts1_lag_3         12.772186
return_correlation_ts2_lag_1         12.877910
return_correlation_ts2_lag_2         13.704253
return_correlation_ts2_lag_3         12.391970
sqreturn_auto

In [3]:
mu11 = 0.5689102
mu12 = 0.1308214 
mu21 = 0.6956711 
mu22 = 0.1349717
sigma11 = -3.3933278
sigma12 = -1.2515663 
sigma21 = -1.3669886
sigma22 = -2.9761180   
print(params)

[ 1.62146449  0.51486605  0.80366418  0.07960476 -8.9826998  -1.28491736
 -1.40697841 -2.67024584]
